In [13]:
import re
import pandas as pd
import numpy as np
import seaborn as snb
import matplotlib.pyplot as plt
from pdfminer.pdfinterp import PDFResourceManager , PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import io as IO
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

%matplotlib inline

In [3]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    strio = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, strio, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = strio.getvalue()

    fp.close()
    device.close()
    strio.close()
    return text

In [4]:
lone=convert_pdf_to_txt('tesla.pdf')

In [5]:
lone

' tsla-10k_20181231.htm\n\nUNITED STATES\n\nSECURITIES AND EXCHANGE COMMISSION\n\nWashington, D.C. 20549\n\nFORM 10-K\n\n(Mark One)\n\n\u0a34 ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended December 31, 2018 \n\nOR\n\nਲ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from                      to                      \n\nCommission File Number: 001-34756\n\nTesla, Inc.\n\n(Exact name of registrant as specified in its charter)\n\nDelaware\n(State or other jurisdiction of\n\n91-2197729\n(I.R.S. Employer\n\nincorporation or organization)\n\nIdentification No.)\n\n3500 Deer Creek Road\n\nPalo Alto, California\n(Address of principal executive offices)\n\n94304\n(Zip Code)\n\n(650) 681-5000\n\n(Registrant’s telephone number, including area code)\n\nSecurities registered pursuant to Section 12(b) of the Act:\n\nSecurities registered pursuant to Section 12(g) of th

In [6]:
f=IO.open('xxx.txt','w', encoding="utf8")
f.write(lone)
f.close()

In [7]:
with open('xxx.txt', encoding="utf8") as f:
    clean_cont = f.read().splitlines()

In [8]:
clean_cont

[' tsla-10k_20181231.htm',
 '',
 'UNITED STATES',
 '',
 'SECURITIES AND EXCHANGE COMMISSION',
 '',
 'Washington, D.C. 20549',
 '',
 'FORM 10-K',
 '',
 '(Mark One)',
 '',
 '\u0a34 ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934',
 'For the fiscal year ended December 31, 2018 ',
 '',
 'OR',
 '',
 'ਲ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934',
 'For the transition period from                      to                      ',
 '',
 'Commission File Number: 001-34756',
 '',
 'Tesla, Inc.',
 '',
 '(Exact name of registrant as specified in its charter)',
 '',
 'Delaware',
 '(State or other jurisdiction of',
 '',
 '91-2197729',
 '(I.R.S. Employer',
 '',
 'incorporation or organization)',
 '',
 'Identification No.)',
 '',
 '3500 Deer Creek Road',
 '',
 'Palo Alto, California',
 '(Address of principal executive offices)',
 '',
 '94304',
 '(Zip Code)',
 '',
 '(650) 681-5000',
 '',
 '(Registrant’s telephone number, i

In [9]:
shear=[i.replace('\u0a34','    ') for i in clean_cont ]
shear=[i.replace('ਲ','') for i in shear ]
shear

[' tsla-10k_20181231.htm',
 '',
 'UNITED STATES',
 '',
 'SECURITIES AND EXCHANGE COMMISSION',
 '',
 'Washington, D.C. 20549',
 '',
 'FORM 10-K',
 '',
 '(Mark One)',
 '',
 '     ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934',
 'For the fiscal year ended December 31, 2018 ',
 '',
 'OR',
 '',
 ' TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934',
 'For the transition period from                      to                      ',
 '',
 'Commission File Number: 001-34756',
 '',
 'Tesla, Inc.',
 '',
 '(Exact name of registrant as specified in its charter)',
 '',
 'Delaware',
 '(State or other jurisdiction of',
 '',
 '91-2197729',
 '(I.R.S. Employer',
 '',
 'incorporation or organization)',
 '',
 'Identification No.)',
 '',
 '3500 Deer Creek Road',
 '',
 'Palo Alto, California',
 '(Address of principal executive offices)',
 '',
 '94304',
 '(Zip Code)',
 '',
 '(650) 681-5000',
 '',
 '(Registrant’s telephone number, incl

In [10]:

shear_without_space = [x for x in shear if x != '']
shear_without_space = [x for x in shear_without_space if x != ' ']
shear_without_space = [x for x in shear_without_space if x != '  ']
shear_without_space = [x for x in shear_without_space if x != '   ']
shear_without_space = [x for x in shear_without_space if x != '    ']
shear_without_space

[' tsla-10k_20181231.htm',
 'UNITED STATES',
 'SECURITIES AND EXCHANGE COMMISSION',
 'Washington, D.C. 20549',
 'FORM 10-K',
 '(Mark One)',
 '     ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934',
 'For the fiscal year ended December 31, 2018 ',
 'OR',
 ' TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934',
 'For the transition period from                      to                      ',
 'Commission File Number: 001-34756',
 'Tesla, Inc.',
 '(Exact name of registrant as specified in its charter)',
 'Delaware',
 '(State or other jurisdiction of',
 '91-2197729',
 '(I.R.S. Employer',
 'incorporation or organization)',
 'Identification No.)',
 '3500 Deer Creek Road',
 'Palo Alto, California',
 '(Address of principal executive offices)',
 '94304',
 '(Zip Code)',
 '(650) 681-5000',
 '(Registrant’s telephone number, including area code)',
 'Securities registered pursuant to Section 12(b) of the Act:',
 'Securities regi

In [12]:
backup_without_numbers=[re.sub("[^a-zA-Z]+", " ", s) for s in shear_without_space]
backup_without_numbers

[' tsla k htm',
 'UNITED STATES',
 'SECURITIES AND EXCHANGE COMMISSION',
 'Washington D C ',
 'FORM K',
 ' Mark One ',
 ' ANNUAL REPORT PURSUANT TO SECTION OR d OF THE SECURITIES EXCHANGE ACT OF ',
 'For the fiscal year ended December ',
 'OR',
 ' TRANSITION REPORT PURSUANT TO SECTION OR d OF THE SECURITIES EXCHANGE ACT OF ',
 'For the transition period from to ',
 'Commission File Number ',
 'Tesla Inc ',
 ' Exact name of registrant as specified in its charter ',
 'Delaware',
 ' State or other jurisdiction of',
 ' ',
 ' I R S Employer',
 'incorporation or organization ',
 'Identification No ',
 ' Deer Creek Road',
 'Palo Alto California',
 ' Address of principal executive offices ',
 ' ',
 ' Zip Code ',
 ' ',
 ' Registrant s telephone number including area code ',
 'Securities registered pursuant to Section b of the Act ',
 'Securities registered pursuant to Section g of the Act ',
 'Title of each class',
 'Common Stock par value The NASDAQ Stock Market LLC',
 'Name of each exchange o

MODELLING

In [14]:
vect=CountVectorizer(ngram_range=(1,1),stop_words='english')

In [16]:
dtm=vect.fit_transform(backup_without_numbers)
dtm

<18596x5519 sparse matrix of type '<class 'numpy.int64'>'
	with 93435 stored elements in Compressed Sparse Row format>

In [17]:
pd.DataFrame(dtm.toarray(),columns=vect.get_feature_names())

,aa,aaam,aaamg,ab,abandoned,abandonment,abatement,abatements,abbreviation,abcp,...,yuan,yyyy,zealand,zep,zero,zev,zhejiang,zhengzhou,zip,zurich
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
lda=LatentDirichletAllocation(n_components=10)
lda_dtf = lda.fit_transform(dtm)
lda_dtf

array([[0.03333333, 0.03333333, 0.03333333, ..., 0.7       , 0.03333333,
        0.03333333],
       [0.03333412, 0.03333333, 0.03333333, ..., 0.69999404, 0.03333333,
        0.03333333],
       [0.025     , 0.025     , 0.775     , ..., 0.025     , 0.025     ,
        0.025     ],
       ...,
       [0.025     , 0.025     , 0.02500405, ..., 0.025     , 0.02500204,
        0.02500007],
       [0.025     , 0.02500504, 0.02500342, ..., 0.02500993, 0.02501172,
        0.02500007],
       [0.1       , 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ]])

In [22]:
sorting=np.argsort(lda.components_)[:,::-1]
features=np.array(vect.get_feature_names())

In [31]:
import mglearn
mglearn.tools.print_topics(topics=range(10), feature_names=features,
sorting=sorting, topics_per_chunk=3, n_words=5)


## start from line 43 of the code you are refrencing too

topic 0       topic 1       topic 2       
--------      --------      --------      
new           solar         securities    
llc           energy        information   
delaware      solarcity     filed         
lender        respect       exchange      
bank          systems       commission    


topic 3       topic 4       topic 5       
--------      --------      --------      
shall         agent         tesla         
agreement     administrativerequested     
warehouse     group         treatment     
subi          shall         confidential  
borrower      borrower      vehicles      


topic 6       topic 7       topic 8       
--------      --------      --------      
december      date          value         
revenue       tsla          stock         
year          htm           notes         
million       rate          net           
ended         facility      common        


topic 9       
--------      
term          
costs         
obligations   
liabilities   
t